In [33]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.chains.summarize import load_summarize_chain

from langchain_openai import ChatOpenAI,OpenAI
import langchain 
langchain.debug=True


#Loading the File
loader = PyPDFLoader("4.pdf")
docs=loader.load()
from langchain_text_splitters import CharacterTextSplitter
print(len(docs))

#Initiating a splitter function
text_splitter = CharacterTextSplitter(
    separator="\n", chunk_size=100000, chunk_overlap=100, length_function=len)


#Aggregating loaded doc into one single string
result = '\n'.join(doc.page_content for doc in docs)
print(len(result))

#Splitting the text based on Chunk size of 100000
docs = text_splitter.split_text(result)
from dotenv import load_dotenv
import os

from langchain_core.documents import Document
#Creating a Langchain Document Object for Summarization Chain

temp_doc=[Document(page_content=doc) for doc in docs]
len(docs)
#Assigning the
docs=temp_doc
load_dotenv(override=True)


api_key = os.getenv('open_ai_key')


llm = ChatOpenAI(openai_api_key=api_key,model="gpt-4o-mini", temperature=0)

756
1046434


In [34]:
print(len(docs))

11


In [25]:
import getpass
import os
import time

from pinecone import Pinecone, ServerlessSpec

if not os.getenv("PINECONE_API_KEY"):
    os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter your Pinecone API key: ")

pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)

In [26]:
import time

index_name = "draft"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=3072,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)



import getpass

os.environ["OPENAI_API_KEY"] = api_key

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)
from uuid import uuid4
uuids = [str(uuid4()) for _ in range(len(docs[:100]))]

vector_store.add_documents(documents=docs, ids=uuids)

['afd3533d-8011-4046-baf1-0b8597b36ee9',
 'f5a65039-6e7c-4948-a07c-eb5fe2dc5daa',
 '771aceb3-0538-4d3c-9075-9829ee95cebf',
 'd504babe-7df6-4225-894b-1ba37124a4e9',
 '735e8be8-a16a-4771-814d-311ba1b6c64d',
 'e9498252-1a75-4364-999b-8cefc9622542',
 '4aeb2239-145d-4e63-8127-85297d5dfd2f',
 'e6861a34-9f4d-453c-84c1-08b9447ae2d2',
 'a7ce969d-52ec-4683-9b6f-9bb0364afa88',
 'dc522601-48d2-4bee-b4e2-234d4f1ddf38',
 'e2d9c185-7e25-4601-8732-65be56c33348',
 '08d95c91-39a0-47e9-87c3-34382a3190ac',
 '0c13b3c5-2022-481d-8d1f-dac7b9c67463',
 'e692d76b-45c6-4767-889b-6e87d9ee2e69',
 'ec21ca35-7784-4eea-9a37-c23124dccdc0',
 'f39b2458-bb32-4909-9e60-7d8c3ccd7aa6',
 'fbe39838-5031-490b-984d-8954f0fa7147',
 '80f3c9c3-e5e8-4c2d-a74b-22f8cdca3c8b',
 'd90964fc-f28e-468a-a902-7c18f169c28c',
 'cbb61972-6ad9-4520-9867-f5708f0ce2e1',
 'f7ee00f3-3a0f-4fc1-bd03-d4bdaed7d231',
 '85cb504a-04b1-4efa-9fc4-ce1d759bd559',
 '8b811f99-a1e2-4ba0-93b6-e7f07a92bc93',
 '6b42ef17-804a-4df7-ae4e-3c81a8148f7f',
 '5b86666e-1145-

In [35]:
from langchain.chains import RefineDocumentsChain, LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_community.llms import OpenAI

# This controls how each document will be formatted. Specifically,
# it will be passed to `format_document` - see that function for more
# details.
document_prompt = PromptTemplate(
    input_variables=["page_content"],
    template="{page_content}"
)
document_variable_name = "context"


# Updated prompt for initial processing to generate exactly 20 questions
prompt = PromptTemplate.from_template(
    "Based on the following content, generate exactly 20 questions that will help in drafting a legal document: {context}"
)
initial_llm_chain = LLMChain(llm=llm, prompt=prompt)
initial_response_name = "prev_response"

# Updated prompt for refinement to enhance and finalize the questions, ensuring a total of 20 questions
prompt_refine = PromptTemplate.from_template(
    "Here are the initial 20 questions generated: {prev_response}. "
    "Refine these questions and ensure the final set contains exactly 20 questions based on the following additional context: {context}"
)
refine_llm_chain = LLMChain(llm=llm, prompt=prompt_refine)

# Create the RefineDocumentsChain with updated chains and prompts
chain = RefineDocumentsChain(
    initial_llm_chain=initial_llm_chain,
    refine_llm_chain=refine_llm_chain,
    document_prompt=document_prompt,
    document_variable_name=document_variable_name,
    initial_response_name=initial_response_name,
)


In [36]:
k=chain.run(docs)

[chain/start] [chain:RefineDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RefineDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "context": "WINNING APPEALS \nCOUNCIL ARGUMENTS\n11TH EDITION\nby\nSarah H. Bohr\n2337 SEMINOLE ROAD•ATLANTI CBEACH FL32233\nTELEPHONE 904246-760 3 FACSIMIL E904239-3222\nwww.SarahBohrPublica tions.com\nWINNING APPEALS  COUNCIL\nARGUMENTS\n11th EDITION\nWritten by\nSarah H. Bohr\n©2024\nSarah H. B ohr\nBohr &  Harring ton, L LC\n2337 Seminole Road\nAtlantic B each, F lorida 32233\nTelephone: (904) 246-7603\nFacsimile: (904) 239-3222\nE-mail: sarahhbohr@aol.com\nwww.SarahB ohrPublications.com\nPREFACE\nThis is the Eleventh Edition of Winning Appeals Council Arguments , which includes the major\nchang es in the law since the last edition, published in May 2022.  These include discussion of new\nSocial Security  Ruling s that have been enacted since the prior edition as well as regulatory  chang es\nenacted since 

In [37]:
print(k)

Here is a refined set of 20 questions based on the provided context, ensuring clarity and specificity while addressing the potential errors made by the ALJ in evaluating the claimant's various impairments:

1. Did the ALJ adequately evaluate the functional impact of the claimant's severe obesity on her residual functional capacity (RFC), as required by SSR 96-8p and SSR 19-2p?

2. How did the ALJ's failure to discuss obesity-related limitations in the RFC assessment affect the overall disability determination?

3. Did the ALJ properly consider the claimant's degenerative disc disease, diabetes, and hypertension in conjunction with her obesity, particularly regarding their cumulative effects on her functional capacity?

4. Did the ALJ err in concluding that the claimant could perform her past relevant work as a cloth inspector, given the vocational expert's testimony that this job requires medium exertional levels and frequent overhead reaching?

5. Did the Appeals Council appropriately

In [38]:
with open("output.txt", "w") as file:
    file.write(k)

In [42]:
import easyocr
from pdf2image import convert_from_path
import os

def ocr_pdf(pdf_path, output_dir):
    # Create output directory if it does not exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Convert PDF to images
    images = convert_from_path(pdf_path,poppler_path=r'poppler-23.08.0\Library\bin')
    
    # Initialize EasyOCR reader
    reader = easyocr.Reader(['en'])
    
    # Process each page
    for i, image in enumerate(images):
        # Save image to a temporary file
        temp_image_path = os.path.join(output_dir, f'page_{i + 1}.jpg')
        image.save(temp_image_path, 'JPEG')
        
        # Perform OCR
        results = reader.readtext(temp_image_path)
        
        # Print results
        print(f'--- Page {i + 1} ---')
        for result in results:
            text, confidence = result[1], result[2]
            print(f'Text: {text}, Confidence: {confidence:.2f}')
        
        # Optionally remove the temporary image file
        os.remove(temp_image_path)


pdf_path = '4.pdf'
output_dir = 'output_images'
ocr_pdf(pdf_path, output_dir)


PDFInfoNotInstalledError: Unable to get page count. Is poppler installed and in PATH?